In [ ]:
수집 데이터 기반 적정 가격 예측 AI 모델링 구축

AI 모델 구축

적정가격 산출 AI 모델링 구축을 통해 부동산 정보 격차 해소

AI 모델링 구축을 통해 각 지역별, 매물별 부동산 적정가격을 산출 하고자 함

## Import

In [8]:
from PublicDataReader import TransactionPrice

import xlwings as xw
import pandas as pd
import numpy as np
import time
import requests
import xmltodict
import os

# path setting
if not os.path.isdir('../datasets/mol/'):
    os.mkdir('../datasets/mol/')
    
os.chdir('../datasets/mol/')

In [38]:
url = '	http://apis.data.go.kr/1613000/BldRgstService_v2/getBrFlrOulnInfo'
ENCODE_KEY = "uMbQQ%2F4mk9%2FTevvtkrHnQbGo5PlJLU09kDWb5nZnrzBCElc8Yq4blxorqkKyQlCwBb2i0tTTiwS49c5Bo6UwnA%3D%3D"
DECODE_KEY = 'uMbQQ/4mk9/TevvtkrHnQbGo5PlJLU09kDWb5nZnrzBCElc8Yq4blxorqkKyQlCwBb2i0tTTiwS49c5Bo6UwnA=='

In [ ]:
def get_data(ledger_type,
             sigungu_code,
             bdong_code,
             plat_code=None,
             bun=None,
             ji=None,
             translate=True,
             verbose=False,
             wait_time=30,
             **kwargs):
    """
    건축물대장 정보 조회

    Parameters
    ----------
    ledger_type : str
        건축물대장 유형 (ex. 기본개요, 총괄표제부, 표제부, 층별개요, 부속지번, 전유공용면적, 오수정화시설, 주택가격, 전유부, 지역지구구역, 소유자)
    sigungu_code : str
        시군구 코드 (ex. 11110)
    bdong_code : str
        법정동 코드 (ex. 1111051500)
    plat_code : str
        대지구분 코드 (대지: 0, 산: 1, 블록: 2)
    bun : str
        번 (ex. 200)
    ji : str
        지 (ex. 5)
    translate : bool
        한글 컬럼명으로 변환 여부 (기본값: True)
    verbose : bool
        진행 상황 출력 여부 (기본값: False)
    wait_time : int
        API 요청 간의 대기 시간 (초) (기본값: 30초)
    **kwargs : dict
        API 요청에 필요한 추가 인자
    """
    try:
        # 건축물대장 유형으로 API URL 선택 (ex. 기본개요, 표제부, 총괄표제부 등)
        url = self.meta_dict.get(ledger_type).get("url")
        # 건축물대장 유형으로 API 컬럼 선택 (ex. 기본개요, 표제부, 총괄표제부 등)
        columns = self.meta_dict.get(ledger_type).get("columns")
    except AttributeError:
        raise AttributeError("건축물대장 유형을 확인해주세요.")

    if ledger_type != '소유자':
        # 서비스키, 행수, 시군구코드, 법정동코드 설정
        params = {
            "serviceKey": requests.utils.unquote(self.service_key),
            "numOfRows": 99999,
            "sigunguCd": sigungu_code,
            "bjdongCd": bdong_code,
        }
        # 본번, 부번 설정 시 Zero Fill 적용
        if bun:
            params.update({"bun": str(bun).zfill(4)})
        if ji:
            params.update({"ji": str(ji).zfill(4)})
        # 플랫코드 설정 시 platGbCd 파라미터 추가
        if plat_code:
            params.update({"platGbCd": plat_code})
    else:
        # 서비스키, 행수, 시군구코드, 법정동코드 설정
        params = {
            "serviceKey": requests.utils.unquote(self.service_key),
            "numOfRows": 99999,
            "sigungu_cd": sigungu_code,
            "bjdong_cd": bdong_code,
        }
        # 본번, 부번 설정 시 Zero Fill 적용
        if bun:
            params.update({"bun": str(bun).zfill(4)})
        if ji:
            params.update({"ji": str(ji).zfill(4)})
        # 플랫코드 설정 시 plat_gb_cd 파라미터 추가
        if plat_code:
            params.update({"plat_gb_cd": plat_code})
    # 선택 파라미터 추가 설정
    params.update(kwargs)
    # 빈 데이터 프레임 생성
    df = pd.DataFrame(columns=columns)
    # API 요청
    res = requests.get(url, params=params, verify=False)
    # 요청 결과 JSON 변환
    res_json = xmltodict.parse(res.text)
    # 응답 키 존재 확인
    if not res_json.get("response"):
        if verbose:
            print(res_json)
        raise Exception("API 요청이 실패했습니다.")
    # 결과코드가 정상이 아닌 경우
    if res_json['response']['header']['resultCode'] != '00':
        error_message = res_json['response']['header']['resultMsg']
        raise Exception(error_message)
    # 요청 행 수
    _numOfRows = res_json['response']['body']['numOfRows']
    # 페이지 번호
    _pageNo = res_json['response']['body']['pageNo']
    # 총 데이터 크기
    _totalCount = res_json['response']['body']['totalCount']
    # 순회해야 하는 페이지 수
    _pageNoCount = int(_totalCount) // int(_numOfRows) + 1
    if verbose:
        print(
            f"""- 요청 행 수: {_numOfRows}\n- 현재 페이지 번호: {_pageNo}\n- 총 행 수: {_totalCount}\n- 총 페이지 수: {_pageNoCount}\n- API 요청 대기시간: {wait_time}초""")
    items = res_json['response']['body']['items']
    if not items:
        if translate:
            return self.translate_columns(pd.DataFrame(columns=columns))
        else:
            return pd.DataFrame(columns=columns)
    data = items['item']
    if isinstance(data, list):
        sub = pd.DataFrame(data)
    elif isinstance(data, dict):
        sub = pd.DataFrame([data])
    df = pd.concat([df, sub], axis=0, ignore_index=True)
    if _pageNoCount > 1:
        if verbose:
            print(f"페이지가 {_pageNoCount}개 있습니다.")
        # 페이지 순회
        for i in range(2, _pageNoCount + 1):
            # 다음 페이지 조회 전 대기
            time.sleep(wait_time)
            if verbose:
                print(f"page {i} / {_pageNoCount} 요청")
            params['pageNo'] = i
            # API 요청
            res = requests.get(url, params=params, verify=False)
            # 요청 결과 JSON 변환
            res_json = xmltodict.parse(res.text)
            # 응답 키 존재 확인
            if not res_json.get("response"):
                if verbose:
                    print(res_json)
                raise Exception("API 요청이 실패했습니다.")
            # 결과코드가 정상이 아닌 경우
            if res_json['response']['header']['resultCode'] != '00':
                error_message = res_json['response']['header']['resultMsg']
                raise Exception(error_message)
            items = res_json['response']['body']['items']
            if not items:
                if translate:
                    return self.translate_columns(pd.DataFrame(columns=columns))
                else:
                    return pd.DataFrame(columns=columns)
            data = items['item']
            if isinstance(data, list):
                sub = pd.DataFrame(data)
            elif isinstance(data, dict):
                sub = pd.DataFrame([data])
            df = pd.concat([df, sub], axis=0, ignore_index=True)
    # 컬럼명 한글로 변경
    if translate:
        df = self.translate_columns(df)
    return df

In [62]:
params = {
        'ServiceKey': DECODE_KEY,
        'bjdongCd' : 10300,
        'platGbCd': 0,
        'bun': '0012',
        'ji': '0000',
        'numOfRows': '1000',
        'pageNo': '1',
        'sigunguCd': '11680',
}

In [63]:
r = requests.get(url, params=params)

In [64]:
res_json = xmltodict.parse(r.text)

In [65]:
change_dict ={
            'bjdongCd': '법정동코드',
            'bldNm': '건물명',
            'block': '블록',
            'bun': '번',
            'bylotCnt': '외필지수',
            'crtnDay': '생성일자',
            'guyukCd': '구역코드',
            'guyukCdNm': '구역코드명',
            'ji': '지',
            'jiguCd': '지구코드',
            'jiguCdNm': '지구코드명',
            'jiyukCd': '지역코드',
            'jiyukCdNm': '지역코드명',
            'lot': '로트',
            'mgmBldrgstPk': '관리건축물대장PK',
            'mgmUpBldrgstPk': '관리상위건축물대장PK',
            'naBjdongCd': '새주소법정동코드',
            'naMainBun': '새주소본번',
            'naRoadCd': '새주소도로코드',
            'naSubBun': '새주소부번',
            'naUgrndCd': '새주소지상지하코드',
            'newPlatPlc': '도로명대지위치',
            'platGbCd': '대지구분코드',
            'platPlc': '대지위치',
            'regstrGbCd': '대장구분코드',
            'regstrGbCdNm': '대장구분코드명',
            'regstrKindCd': '대장종류코드',
            'regstrKindCdNm': '대장종류코드명',
            'rnum': '순번',
            'sigunguCd': '시군구코드',
            'splotNm': '특수지명',
            'archArea': '건축면적',
            'atchBldArea': '부속건축물면적',
            'atchBldCnt': '부속건축물수',
            'bcRat': '건폐율',
            'engrEpi': 'EPI점수',
            'engrGrade': '에너지효율등급',
            'engrRat': '에너지절감율',
            'etcPurps': '기타용도',
            'fmlyCnt': '가구수',
            'gnBldCert': '친환경건축물인증점수',
            'gnBldGrade': '친환경건축물등급',
            'hhldCnt': '세대수',
            'hoCnt': '호수',
            'indrAutoArea': '옥내자주식면적',
            'indrAutoUtcnt': '옥내자주식대수',
            'indrMechArea': '옥내기계식면적',
            'indrMechUtcnt': '옥내기계식대수',
            'itgBldCert': '지능형건축물인증점수',
            'itgBldGrade': '지능형건축물등급',
            'mainBldCnt': '주건축물수',
            'mainPurpsCd': '주용도코드',
            'mainPurpsCdNm': '주용도코드명',
            'newOldRegstrGbCd': '신구대장구분코드',
            'newOldRegstrGbCdNm': '신구대장구분코드명',
            'oudrAutoArea': '옥외자주식면적',
            'oudrAutoUtcnt': '옥외자주식대수',
            'oudrMechArea': '옥외기계식면적',
            'oudrMechUtcnt': '옥외기계식대수',
            'platArea': '대지면적',
            'pmsDay': '허가일',
            'pmsnoGbCd': '허가번호구분코드',
            'pmsnoGbCdNm': '허가번호구분코드명',
            'pmsnoKikCd': '허가번호기관코드',
            'pmsnoKikCdNm': '허가번호기관코드명',
            'pmsnoYear': '허가번호년',
            'stcnsDay': '착공일',
            'totArea': '연면적',
            'totPkngCnt': '총주차수',
            'useAprDay': '사용승인일',
            'vlRat': '용적률',
            'vlRatEstmTotArea': '용적률산정연면적',
            'dongNm': '동명칭',
            'emgenUseElvtCnt': '비상용승강기수',
            'etcRoof': '기타지붕',
            'etcStrct': '기타구조',
            'grndFlrCnt': '지상층수',
            'heit': '높이',
            'mainAtchGbCd': '주부속구분코드',
            'mainAtchGbCdNm': '주부속구분코드명',
            'rideUseElvtCnt': '승용승강기수',
            'roofCd': '지붕코드',
            'roofCdNm': '지붕코드명',
            'rserthqkAblty': '내진 능력',
            'rserthqkDsgnApplyYn': '내진 설계 적용 여부',
            'strctCd': '구조코드',
            'strctCdNm': '구조코드명',
            'totDongTotArea': '총동연면적',
            'ugrndFlrCnt': '지하층수',
            'area': '면적',
            'areaExctYn': '면적제외여부',
            'flrGbCd': '층구분코드',
            'flrGbCdNm': '층구분코드명',
            'flrNo': '층번호',
            'flrNoNm': '층번호명',
            'atchBjdongCd': '부속법정동코드',
            'atchBlock': '부속블록',
            'atchBun': '부속번',
            'atchEtcJibunNm': '부속기타지번명',
            'atchJi': '부속지',
            'atchLot': '부속로트',
            'atchPlatGbCd': '부속대지구분코드',
            'atchRegstrGbCd': '부속대장구분코드',
            'atchRegstrGbCdNm': '부속대장구분코드명',
            'atchSigunguCd': '부속시군구코드',
            'atchSplotNm': '부속특수지명',
            'exposPubuseGbCd': '전유공용구분코드',
            'exposPubuseGbCdNm': '전유공용구분코드명',
            'hoNm': '호명칭',
            'capaLube': '용량(루베)',
            'capaPsper': '용량(인용)',
            'etcMode': '기타형식',
            'modeCd': '형식코드',
            'modeCdNm': '형식코드명',
            'unitGbCd': '단위구분코드',
            'unitGbCdNm': '단위구분코드명',
            'hsprc': '주택가격',
            'etcJijigu': '기타지역지구구역',
            'jijiguCd': '지역지구구역코드',
            'jijiguCdNm': '지역지구구역코드명',
            'jijiguGbCd': '지역지구구역구분코드',
            'jijiguGbCdNm': '지역지구구역구분코드명',
            'reprYn': '대표여부',
            # 건축소유자정보
            'mgm_bldrgst_pk': '관리건축물대장PK',
            'sigungu_cd': '시군구코드',
            'sigungu_nm': '시군구명',
            'bjdong_cd': '법정동코드',
            'bjdong_nm': '법정동명',
            'plat_gb_cd': '대지구분코드',
            'plat_gb_nm': '대지구분명',
            'bun': '번',
            'ji': '지',
            'splot_nm': '특수지명',
            'block': '블록',
            'lot': '로트',
            'na_plat_plc': '새주소대지위치',
            'na_road_cd': '새주소도로코드',
            'na_bjdong_cd': '새주소법정동코드',
            'na_ugrnd_cd': '새주소지상지하코드',
            'na_ugrnd_nm': '새주소지상지하명',
            'na_main_bun': '새주소본번',
            'na_sub_bun': '새주소부번',
            'regstr_gb_cd': '대장구분코드',
            'regstr_gb_nm': '대장구분명',
            'regstr_kind_cd': '대장종류코드',
            'regstr_kind_nm': '대장종류명',
            'bld_nm': '건물명',
            'dong_nm': '동명칭',
            'ho_nm': '호명칭',
            'area': '면적',
            'own_gb_cd': '소유구분코드',
            'own_gb_nm': '소유구분명',
            'jm_gb_cd': '주민구분코드',
            'jm_gb_nm': '주민구분명',
            'nm': '성명',
            'quota1': '지분1',
            'quota2': '지분2',
            'ownsh_quota': '소유권지분',
            'chang_caus_day': '변동원인일',
            'loc_sigungu_cd': '소재지시군구코드',
            'loc_sigungu_nm': '소재지시군구명',
            'loc_bjdong_cd': '소재지법정동코드',
            'loc_bjdong_nm': '소재지법정동명',
            'loc_detl_addr': '소재지상세주소',
            'na_loc_plat_plc': '새주소소재지대지위치',
            'na_loc_road_cd': '새주소소재지도로코드',
            'na_loc_bjdong_cd': '새주소소재지법정동코드',
            'na_loc_detl_addr': '새주소소재지상세주소',
            'na_loc_ugrnd_cd': '새주소소재지지상지하코드',
            'na_loc_ugrnd_nm': '새주소소재지지상지하명',
            'na_loc_main_bun': '새주소소재지본번',
            'jmno': '주민번호',
            'na_loc_sub_bun': '새주소소재지부번'}

In [69]:
res_json['response'].keys()

dict_keys(['header', 'body'])

In [66]:
df = pd.DataFrame(res_json['response']['body']['items']['item'])
df.rename(columns = change_dict, inplace = True)

In [67]:
df.iloc[:, :18]

,면적,면적제외여부,법정동코드,건물명,블록,번,생성일자,동명칭,기타용도,기타구조,층구분코드,층구분코드명,층번호,층번호명,지,로트,주부속구분코드,주부속구분코드명
0,271.63,None,10300,None,None,0012,20171206,None,교육연구및복지시설,연와조,20,지상,1.0,1층,0000,None,0,주건축물
1,264.43,None,10300,None,None,0012,20171206,None,교육연구및복지시설,연와조,20,지상,2.0,2층,0000,None,0,주건축물
2,125.87,None,10300,대한예수교장로회 한울교회,None,0012,20181117,None,문화및집회시설,철근콘크리트조,10,지하,1.0,지하1층,0000,None,0,주건축물
3,243.59,None,10300,대한예수교장로회 한울교회,None,0012,20181117,None,문화및집회시설,철근콘크리트조,20,지상,1.0,1층,0000,None,0,주건축물
4,243.59,None,10300,대한예수교장로회 한울교회,None,0012,20181117,None,문화및집회시설,철근콘크리트조,20,지상,2.0,2층,0000,None,0,주건축물
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,425.19,None,10300,대치아파트214동,None,0012,20190530,214,아파트,철근콘크리트조,20,지상,2.0,2층,0000,None,0,주건축물
96,425.19,None,10300,대치아파트214동,None,0012,20190530,214,아파트,철근콘크리트조,20,지상,3.0,3층,0000,None,0,주건축물
97,425.19,None,10300,대치아파트214동,None,0012,20190530,214,아파트,철근콘크리트조,20,지상,5.0,5층,0000,None,0,주건축물
98,425.19,None,10300,대치아파트214동,None,0012,20190530,214,아파트,철근콘크리트조,20,지상,6.0,6층,0000,None,0,주건축물


In [61]:
df.iloc[:, 18:]

,주용도코드,주용도코드명,관리건축물대장PK,새주소법정동코드,새주소본번,새주소도로코드,새주소부번,새주소지상지하코드,도로명대지위치,대지구분코드,대지위치,순번,시군구코드,특수지명,구조코드,구조코드명
0,10999,기타교육연구시설,11680-7004,None,0.0,None,0.0,0,None,0,서울특별시 강남구 개포동 12번지,1,11680,None,11,벽돌구조
1,10999,기타교육연구시설,11680-7004,None,0.0,None,0.0,0,None,0,서울특별시 강남구 개포동 12번지,2,11680,None,11,벽돌구조
2,06101,교회,11680-7003,None,0.0,None,0.0,0,None,0,서울특별시 강남구 개포동 12번지,3,11680,None,21,철근콘크리트구조
3,06101,교회,11680-7003,None,0.0,None,0.0,0,None,0,서울특별시 강남구 개포동 12번지,4,11680,None,21,철근콘크리트구조
4,06101,교회,11680-7003,None,0.0,None,0.0,0,None,0,서울특별시 강남구 개포동 12번지,5,11680,None,21,철근콘크리트구조
5,04402,사무소,11680-7005,10301,5.0,116804166040,0.0,0,서울특별시 강남구 개포로109길 5,0,서울특별시 강남구 개포동 12번지,6,11680,None,21,철근콘크리트구조
6,11999,기타노유자시설,11680-7005,10301,5.0,116804166040,0.0,0,서울특별시 강남구 개포로109길 5,0,서울특별시 강남구 개포동 12번지,7,11680,None,21,철근콘크리트구조
7,11999,기타노유자시설,11680-7005,10301,5.0,116804166040,0.0,0,서울특별시 강남구 개포로109길 5,0,서울특별시 강남구 개포동 12번지,8,11680,None,21,철근콘크리트구조
8,11999,기타노유자시설,11680-7005,10301,5.0,116804166040,0.0,0,서울특별시 강남구 개포로109길 5,0,서울특별시 강남구 개포동 12번지,9,11680,None,21,철근콘크리트구조
9,11999,기타노유자시설,11680-7005,10301,5.0,116804166040,0.0,0,서울특별시 강남구 개포로109길 5,0,서울특별시 강남구 개포동 12번지,10,11680,None,21,철근콘크리트구조


In [43]:
from PublicDataReader import BuildingLedger

service_key = "uMbQQ%2F4mk9%2FTevvtkrHnQbGo5PlJLU09kDWb5nZnrzBCElc8Yq4blxorqkKyQlCwBb2i0tTTiwS49c5Bo6UwnA%3D%3D"
api = BuildingLedger(DECODE_KEY)

df = api.get_data(
    ledger_type="기본개요", 
    sigungu_code="41135", 
    bdong_code="11000", 
    bun="542", 
    ji="",
)
df.head()

ConnectionError: HTTPConnectionPool(host='apis.data.go.kr', port=80): Max retries exceeded with url: /1613000/BldRgstService_v2/getBrBasisOulnInfo?serviceKey=uMbQQ%2F4mk9%2FTevvtkrHnQbGo5PlJLU09kDWb5nZnrzBCElc8Yq4blxorqkKyQlCwBb2i0tTTiwS49c5Bo6UwnA%3D%3D&numOfRows=99999&sigunguCd=41135&bjdongCd=11000&bun=0542&pageNo=4 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001F2515290A0>: Failed to establish a new connection: [WinError 10060] 연결된 구성원으로부터 응답이 없어 연결하지 못했거나, 호스트로부터 응답이 없어 연결이 끊어졌습니다'))

* GET DATA

In [62]:
url = 'http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTradeDev?_wadl&type=xml'
#url = 'http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTradeDev'
ENCODE_KEY = "uMbQQ%2F4mk9%2FTevvtkrHnQbGo5PlJLU09kDWb5nZnrzBCElc8Yq4blxorqkKyQlCwBb2i0tTTiwS49c5Bo6UwnA%3D%3D"
DECODE_KEY = 'uMbQQ/4mk9/TevvtkrHnQbGo5PlJLU09kDWb5nZnrzBCElc8Yq4blxorqkKyQlCwBb2i0tTTiwS49c5Bo6UwnA=='

In [65]:
params = {
        #'ServiceKey': ENCODE_KEY,
        'ServiceKey': DECODE_KEY,
        'numOfRows' : 999999,
        'LAWD_CD': '11110',
        'DEAL_YMD': '201512',
}

r = requests.get(url, params=params, verify = False)

In [66]:
res_json = xmltodict.parse(r.text)

In [67]:
res_json.keys()

dict_keys(['application'])

In [68]:
res_json['application'].keys()

dict_keys(['@xmlns', '@xmlns:xs', 'grammars', 'resources'])

In [72]:
res_json['application']['resources']

{'@base': 'http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc',
 'resource': {'@path': '/',
  'resource': [{'@path': 'getRTMSDataSvcAptRent',
    'method': {'@name': 'GET',
     'request': {'param': [{'@name': 'LAWD_CD',
        '@style': 'query',
        '@type': 'xs:string'},
       {'@name': 'DEAL_YMD', '@style': 'query', '@type': 'xs:string'},
       {'@name': 'pageNo',
        '@style': 'query',
        '@default': '1',
        '@type': 'xs:int'},
       {'@name': 'numOfRows',
        '@style': 'query',
        '@default': '10',
        '@type': 'xs:int'}]},
     'response': {'representation': [{'@mediaType': 'application/xml'},
       {'@mediaType': 'application/json'}]}}},
   {'@path': 'getRTMSDataSvcAptRentDev',
    'method': {'@name': 'GET',
     'request': {'param': [{'@name': 'LAWD_CD',
        '@style': 'query',
        '@type': 'xs:string'},
       {'@name': 'DEAL_YMD', '@style': 'query', '@type': 'xs:string'},
       {'@name': 'pageNo',
        

In [70]:
if res_json['response']['header']['resultCode'] != '00':
    error_message = res_json['response']['header']['resultMsg']
    raise Exception(error_message)
items = res_json['response']['body']['items']
if not items:
    return pd.DataFrame(columns=columns)
data = items['item']
if isinstance(data, list):
    sub = pd.DataFrame(data)
elif isinstance(data, dict):
    sub = pd.DataFrame([data])
df = pd.concat([df, sub], axis=0, ignore_index=True)

KeyError: 'response'

In [73]:


tr_api = TransactionPrice(DECODE_KEY)

# 단일 월 조회
df = tr_api.get_data(
    property_type="아파트",
    trade_type="매매",
    sigungu_code="11650",
    year_month="202212",
)

# 기간 내 조회
df = tr_api.get_data(
    property_type="아파트",
    trade_type="매매",
    sigungu_code="11650",
    start_year_month="202201",
    end_year_month="202212",
)

df.tail()

Exception: SERVICE KEY IS NOT REGISTERED ERROR.